# Recurrent Neural Networks Tutorial, Part 2 – Implementing a Language Modeling RNN with Python and Theano

In [1]:
import csv
import itertools
import operator
import numpy as np
import matplotlib.pyplot as plt
import theano
import theano.tensor as T
import nltk
from utils import softmax

# Display plots inline
%matplotlib inline

This is the second part of the Recurrent Neural Network Tutorial. [The first part can be found here](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/). In this part we will implement a full Recurrent Neural Network from Scratch using Python. Then we will optimize our implementation using [Theano](http://deeplearning.net/software/theano/), a Python library that can make efficient use of a GPU. **[The full code is also available on Github](https://github.com/dennybritz/rnn-tutorial-rnnlm/)**. I may skip over some boilerplate code that is not essentialy to understanding Recurrent Neural Networks in this post, but it's all on Github.

### Language Modeling

Our goal is to build a [Language Model](https://en.wikipedia.org/wiki/Language_model) using a Recurrent Neural Network. Here's what that means. Let's say we have sentence consisting of $m$ words. Then a Language Model allows us to predict probability of observing that sentence in the data we care about as:

$
\begin{align}
P(w_1,...,w_m) = \prod_{i=1}^{m}P(w_i \mid w_1,..., w_{i-1}) 
\end{align}
$

In words, the probability of that sentence is the product of probabilities of each word given that words that came before it. The probability of the sentence "He went to buy some chocolate" would be the probability of "chocolate" given "He went to buy some", multiplied by the probability of "some" given "He went to buy", and so on. 

But what is this useful for? Why would we want to assign a probability to observing a sentence?

First, it can be used as a scoring mechanism. For example, if you have a Machine Translation system that generates several possible translations of a sentence you can use the model to find the most probable sentence, which is likely to be a grammatically correct one. Similar scoring happens in speech recognition systems. You have several sentence candidates and want to figure out which candidate is most likely to be correct.

But solving the Language Modeling problem has another cool side effect. Because we can predict the probability of a word given the words that came before it we are able to generate new text. We have a *generative model*. Given an existing sequence of words we simply find the word that has the highest probability to occur next, take that, and repeat the process until we have a full sentence.

Andrew Karparthy [has a great post](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) that demonstrates what languages models are capable of. His models are trained on single characters as opposed to full words, and they can generate anything from Shakespeare to Linux Code.

Note that in the above equation the probability of each words depends on **all** previous words. In practice, many types of models have a hard time with such long-term dependencies due to computational or memory constraints. As a result they typically limit themselves to looking at only a few of the previous words. RNNs can in theory capture such long-term dependencies, but in practice it's a bit more complex. We'll explore why that is later and also look at some of the solutions.

### Training Data and Preprocessing

To train our language model we need text to learn from. Fortunately we don't need any labels for our training data because we can generate them ourselves (see below). I downloaded 15,000 reddit with more than 200 characters from a [dataset available on Google's BigQuery](https://bigquery.cloud.google.com/table/fh-bigquery:reddit_comments.2015_08). So our model will be talking like reddit commenters! As with all Machine Learning tasks we need to do some pre-processing to get the data into a format our Neural Network can understand:

#### 1. Tokenize Text

All we have is raw text, but we want to make predict on a per-word basis and generate sentences. That means we must *tokenize* our comments into sentences and the sentences into words. We could just split each of the comments by spaces, but that wouldn't handle punctuation properly. The sentence "He left!"  should be 3 tokens, "He", "left" and "!". So we'll use [NLTK's](http://www.nltk.org/) `word_tokenize` method, which does most of the hard work for us.

#### 2. Remove Infrequent words

Most words in our text will only appear one or two times. It's a good idea to remove these infrequent words for two rasons. First, having a huge vocabulary will make our model slow to train (we'll talk about why that is later). Secondly, because we don't have a lot of contextual examples for these infrequent words we probably wouldn't be able to learn how to use them correctly anyway. That's quite similar to how humans learn. To really understand how to use a word appropriately it's good to see it in several different contexts.

In the code below we limit our vocabulary to the `vocabulary_size` most common words (I set to 2500, but feel free to change). All words that we don't include in our vocabulary will be replaced by "UNKNOWN_TOKEN" in a sentence. For example, if we don't include the word "nonlinearities" in our vocabulary the sentence "nonlineraties are important in neural networks" would become "UNKNOWN_TOKEN are important in Neural Networks". The word "UNKNOWN_TOKEN" will become part of our vocabulary and we will predict it just like any other word. After we generate text we could "UNKNOWN_TOKEN" again, for example with a randomly sampled word that wasn't in our vocabulary. 

#### 3. Prepend special start and end tokens

Besides predicting the next word given the previous words we also want to predict which words are likely to start and end a sentence. To do this we prepend a special `SENTENCE_START` token to each sentence, and similarly append a special `SENTENCE_END` token to each sentence. That allows us to ask: Given that the first word is "SENTENCE_START", what is the likely next word (the actual first word of the sentence)?


#### 4. Build training data

The input to our Recurrent Neural Networks are vectors, not strings. Thus, we create a mapping between words and indices, `index_to_word`, and `word_to_index`. For example, "friendly" may be at index `2001`. A training example $x$ could then be `[0, 179, 341, 416]`, where 999 is `SENTENCE_START`. The corresponding training label $y$  would be `[179, 341, 416, 1]`, which is just the x vector shifted by one position with the last element being the `SENTENCE_END` token. In other words, the correct prediction for word `179` above would be `341`, the actual next word. 

In [58]:
vocabulary_size = 2500
unknown_token = "UNKNOWN_TOKEN"
sentence_start_token = "SENTENCE_START"
sentence_end_token = "SENTENCE_END"

# Read the data and append SENTENCE_START and SENTENCE_END tokens
print "Reading CSV file..."
with open('data/reddit-comments-2015-08.csv', 'rb') as f:
    reader = csv.reader(f, skipinitialspace=True)
    reader.next()
    # Split full comments into sentences
    sentences = itertools.chain(*[nltk.sent_tokenize(x[0].decode('utf-8').lower()) for x in reader])
    # Append SENTENCE_START and SENTENCE_END
    sentences = ["%s %s %s" % (sentence_start_token, x, sentence_end_token) for x in sentences]
print "Parsed %d sentences." % (len(sentences))
    
# Tokenize the sentences into words
tokenized_sentences = [nltk.word_tokenize(sent) for sent in sentences]

# Count the word frequencies
word_freq = nltk.FreqDist(itertools.chain(*tokenized_sentences))
print "Found %d unique words tokens." % len(word_freq.items())

# Get the most common words and build index_to_word and word_to_index vectors
vocab = word_freq.most_common(vocabulary_size-1)
index_to_word = [x[0] for x in vocab]
index_to_word.append(unknown_token)
word_to_index = dict([(w,i) for i,w in enumerate(index_to_word)])

print "The least frequent word in our vocabulary is '%s' and appeared %d times." % (vocab[-1][0], vocab[-1][1])

# Replace all words not in our vocabulary with the unknown token
for i, sent in enumerate(tokenized_sentences):
    tokenized_sentences[i] = [w if w in word_to_index else unknown_token for w in sent]

print "\nExample sentence: '%s'" % sentences[0]
print "\nExample sentence after Pre-processing: '%s'" % tokenized_sentences[0]

Reading CSV file...
Parsed 79170 sentences.
Found 65751 unique words tokens.
The least frequent word in our vocabulary is 'studies' and appeared 43 times.

Example sentence: 'SENTENCE_START i joined a new league this year and they have different scoring rules than i'm used to. SENTENCE_END'

Example sentence after Pre-processing: '[u'SENTENCE_START', u'i', 'UNKNOWN_TOKEN', u'a', u'new', u'league', u'this', u'year', u'and', u'they', u'have', u'different', 'UNKNOWN_TOKEN', u'rules', u'than', u'i', u"'m", u'used', u'to', u'.', u'SENTENCE_END']'


In [59]:
# Create the training data
X_train = np.asarray([[word_to_index[w] for w in sent[:-1]] for sent in tokenized_sentences])
y_train = np.asarray([[word_to_index[w] for w in sent[1:]] for sent in tokenized_sentences])

In [73]:
# Print an training data example
x_example, y_example = X_train[17], y_train[17]
print "Training Example: \n"
print "x:\n%s\n%s" % (" ".join([index_to_word[x] for x in x_example]), x_example)
print "\ny\n%s\n%s" % (" ".join([index_to_word[x] for x in y_example]), y_example)

Training Example: 

x:
SENTENCE_START what are n't you understanding about this ? !
[0, 51, 27, 16, 10, 856, 53, 25, 34, 69]

y
what are n't you understanding about this ? ! SENTENCE_END
[51, 27, 16, 10, 856, 53, 25, 34, 69, 1]


##### Building the RNN

For a general refresher on RNNs check out the [first part of the tutorial](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/). Let's get concrete and see what the RNN for our language model looks like:

![](http://www.wildml.com/wp-content/uploads/2015/09/rnn.jpg)

In our case, the input $x$ will be a sequence of words (just like the example printed above) and each $x_i$ is a single word. But there's one more thing: Because of the way matrix multiplication works we can't just give a word index (like 36) as input. Instead, we represent each word as a *one-hot vector* of size `vocabulary_size`. For example, the word with index 36 would be a vector of all 0's and a 1 at position 36. So, each $x_i$ will become a vector, and $x$ will be a matrix, with each row representing a word. We'll perform this transformation in our Neural Network code instead of doing it in the pre-processing. The output of our network $o$ has a similar format. Each $o_t$ is a vector of `vocabulary_size` elements, and each element represents the probability of that word being the next word in the sentence.

Let's quickly recap the equations for the RNN from the first part of the tutorial:

$
\begin{align}
s_t &= \tanh(Ux_t + Ws_{t-1}) \\
o_t &= \mathrm{softmax}(Vs_t)
\end{align}
$

I always find it useful to write down the dimensions of the matrices and vectors we care about. Let's assume we pick a vocabulary size $L = 2500$ and a hidden layer size $H = 200$. You can think of the hidden layer size as the "memory" of our network. Making it bigger allows us to learn more complex patterns, but also results in additional computation. Then we have:

$
\begin{align}
x_t & \in \mathbb{R}^{2500} \\
o_t & \in \mathbb{R}^{2500} \\
s_t & \in \mathbb{R}^{200} \\
U & \in \mathbb{R}^{200 \times 2500} \\
V & \in \mathbb{R}^{2500 \times 200} \\
W & \in \mathbb{R}^{200 \times 200} \\
\end{align}
$

This gives us some useuful information. Remember that $U,V$ and $W$ are the parameters of our network we need to learn from data. Thus, we need to learn a total of $2HL + H^2$ parameters. In the above example that's 1,040,000. The above also shows us the slowest operation of our network. Note that because $x_t$ is a one-hot vector, multiplying it with $U$ is essentially the same as selecting a column of U, so we don't need to perform the full multiplication. Then, the biggest matrix multiplication in our network is $Vs_t$. That's why we want to keep our vocabulary size small, if possible.

Armed with this, let's start the implementation of the above.

#### Initialization

Let's start by defining a RNN class an initializing our parameters. I'm calling this class `RNNNumpy` because we will also implement a Theano version later on. Initializing the parameters $U,V$ and $W$ is a bit tricky. We can't just initialize them to 0's because that'll result in symmetric calculations of all our neurons. So we need to initialize them randomly. Because Proper initialization has been found to have an impact on training results there is lot of research in this area. It turns out that the best initialization depends on which activation function you are using ($\tanh$ in our case). One such [recommended](http://jmlr.org/proceedings/papers/v9/glorot10a/glorot10a.pdf) initialization is to initialize the weights randomly in the interval from $\left[-\frac{1}{\sqrt{n}}, \frac{1}{\sqrt{n}}\right]$ where $n$ is the number of incoming connections.

In [100]:
class RNNNumpy:
    
    def __init__(self, word_dim, hidden_dim=100, bptt_truncate=4):
        # Assign instance variables
        self.word_dim = word_dim
        self.hidden_dim = hidden_dim
        self.bptt_truncate = bptt_truncate
        # Randomly initialize the network parameters
        self.U = np.random.uniform(-np.sqrt(1/word_dim), np.sqrt(1/word_dim), (hidden_dim, word_dim))
        self.V = np.random.uniform(-np.sqrt(1/hidden_dim), np.sqrt(1/hidden_dim), (word_dim, hidden_dim))
        self.W = np.random.uniform(-np.sqrt(1/hidden_dim), np.sqrt(1/hidden_dim), (hidden_dim, hidden_dim))
        

In the above, `word_dim` is the size of our vocabulary, and `hidden_dim` is the size of our hidden layer. Don't worry about the `bptt_truncate` parameter for now, we'll explain what that is later. 

#### Forward Propagation

Next, let's implement the forward propagation, as defined by our equations above:

In [101]:
    def forward_propagation(self, x):
        # The total number of time steps
        T = len(x)
        # During forward propagation we save all hidden states in s becausewe need them later.
        # We add one additional element for the initial hidden, which we set to 0
        s = np.zeros((T + 1, self.hidden_dim))
        s[-1] = np.zeros(self.hidden_dim)
        # The outputs at each time step. Again, we save them for later.
        o = np.zeros((T, self.word_dim))
        # For each time step...
        for t in np.arange(T):
            # Note that we are indxing U by x[t]. This is the same as multiplying U with a one-hot vector.
            s[t] = np.tanh(self.U[:,x[t]] + self.W.dot(s[t-1]))
            o[t] = softmax(self.V.dot(s[t]))
        return [o, s]

In the code above we not only return the calculated outputs, but also the calculated hidden states. We will need them later on to calculate the gradients and by returning them here we avoid computing them again. Each $o_t$ is a vector of probabilities for each word in our vocabulary. But sometimes, for example when generating text, all we want is the word with the highest probability. We call this function `predict`:

In [102]:
    def predict(self, x):
        # Perform forward propagation and return index of the highest score
        o, s = self.forward_propagation(x)
        return np.argmax(o, axis=1)

#### Calculating the Loss

In [98]:
    def calculate_loss(self, X, y):
        # We accumulate the total loss in L
        L = 0
        for i in np.arange(len(y)):
            x_i, y_i = X[i], y[i]
            o, s = self.forward_propagation(x_i)
            L += -1 * np.sum(np.log(o[np.arange(len(y_i)), y_i]))
        return L
    
    def calculate_mean_loss(self, X, y):
        # Divide calculate_loss by the number of words
        num_words = np.sum([len(y_i) for y_i in y])
        return self.calculate_loss(X,y)/float(num_words)   
            
    def bptt(self, x, y):
        T = len(y)
        # Perform forward propagation
        o, s = self.forward_propagation(x)
        # We accumulate the gradients in these variables
        dLdU = np.zeros(self.U.shape)
        dLdV = np.zeros(self.V.shape)
        dLdW = np.zeros(self.W.shape)
        delta_o = o
        delta_o[np.arange(len(y)), y] -= 1.
        # For each output backwards...
        for t in np.arange(T)[::-1]:
            dLdV += np.outer(delta_o[t], s[t].T)
            # Initial delta calculation
            delta_t = self.V.T.dot(delta_o[t]) * (1 - (s[t] ** 2))
            # Backpropagation through time (for at most self.bptt_truncate steps)
            for bptt_step in np.arange(max(0, t-self.bptt_truncate), t+1)[::-1]:
                # print "Backpropagation step t=%d bptt step=%d " % (t, bptt_step)
                dLdW += np.outer(delta_t, s[bptt_step-1])              
                dLdU[:,x[bptt_step]] += delta_t
                # Update delta for next step
                delta_t = self.W.T.dot(delta_t) * (1 - s[bptt_step-1] ** 2)
        return [dLdU, dLdV, dLdW]
    

In [99]:
def gradient_check(model, x, y, h=0.001, error_threshold=0.01):
    # Overwrite the bptt attribute. We need to backpropagate all the way to get the correct gradient
    model.bptt_truncate = 1000
    # Calculate the gradients using backprop
    bptt_gradients = model.bptt(x, y)
    # List of all parameters we want to chec.
    model_parameters = ['U', 'V', 'W']
    # Gradient check for each parameter
    for pidx, pname in enumerate(model_parameters):
        # Get the actual parameter value from the mode, e.g. model.W
        parameter = operator.attrgetter(pname)(model)
        print "Performing gradient check for parameter %s with size %d." % (pname, np.prod(parameter.shape))
        # Iterate over each element of the parameter matrix, e.g. (0,0), (0,1), ...
        it = np.nditer(parameter, flags=['multi_index'], op_flags=['readwrite'])
        while not it.finished:
            ix = it.multi_index
            # Save the original value so we can reset it later
            original_value = parameter[ix]
            # Estimate the gradient using (f(x+h) - f(x-h))/(2*h)
            parameter[ix] = original_value + h
            gradplus = model.calculate_loss([x],[y])
            parameter[ix] = original_value - h
            gradminus = model.calculate_loss([x],[y])
            estimated_gradient = (gradplus - gradminus)/(2*h)
            parameter[ix] = original_value
            # The gradient for this parameter calculated using backpropagation
            backprop_gradient = bptt_gradients[pidx][ix]
            # calculate The relative error: (|x - y|/(|x| + |y|))
            relative_error = np.abs(backprop_gradient - estimated_gradient)/(np.abs(backprop_gradient) + np.abs(estimated_gradient))
            # If the error is to large fail the gradient check
            if relative_error > error_threshold:
                print "Gradient Check ERROR: parameter=%s ix=%s" % (pname, ix)
                print "+h Loss: %f" % gradplus
                print "-h Loss: %f" % gradminus
                print "Estimated_gradient: %f" % estimated_gradient
                print "Backpropagation gradient: %f" % backprop_gradient
                print "Relative Error: %f" % relative_error
                return 
            it.iternext()
        print "Gradient check for parameter %s passed." % (pname)
            
            
np.random.seed(10)
# To avoid performing millions of expensive calculations we use a smaller vocabulary size for checking.
grad_check_vocab_size = 5
model = RNNNumpy(grad_check_vocab_size, 10)
gradient_check(model, [0,1,2,3], [1,2,3,4])

Performing gradient check for parameter U with size 50.
Gradient check for parameter U passed.
Performing gradient check for parameter V with size 50.
Gradient check for parameter V passed.
Performing gradient check for parameter W with size 100.
Gradient check for parameter W passed.


In [6]:
def train_rnn_with_sgd(model, X_train, y_train, learning_rate=0.0005, print_loss_after=1000, nepoch=1, anneal_after=-1, anneal_factor=0.5):
    # We keep track of the losses so we can plot them later
    losses = []
    for epoch in range(nepoch):
        for i in np.arange(len(y_train)):
            x_i, y_i = X_train[i], y_train[i]
            dLdU, dLdV, dLdW = model.bptt(x_i, y_i)
            model.U -= learning_rate * dLdU
            model.V -= learning_rate * dLdV
            model.W -= learning_rate * dLdW
            if (i % print_loss_after == 0):
                loss = model.calculate_mean_loss(X_train, y_train)
                losses.append(loss)
                print "Loss after epoch=%d i=%d: %f" % (epoch, i,loss)
        # Adjust the learning rate
        if(epoch % anneal_after == 0):
            learning_rate = learning_rate * anneal_factor


In [13]:
# np.random.seed(10)
# model = RNNNumpy(vocabulary_size, 100, bptt_truncate=4)
# losses = train_rnn_with_sgd(model, X_train_onehot[:100], y_train[:100], nepoch=100, learning_rate=0.005, anneal_after=5)

In [189]:
# How long would it take to train a model
np.random.seed(10)
model = RNNNumpy(vocabulary_size, 100, bptt_truncate=4)
# %timeit model.forward_propagation(X_train[3])
# %timeit model.bptt(X_train[3], y_train[3])
o, s = model.forward_propagation(X_train[3])
print o[1,:10]

[ 0.00042649  0.000451    0.00043382  0.00041876  0.00038081  0.00039459
  0.00036433  0.00038306  0.00038365  0.00039345]


In [82]:
class RNNTheano:
    
    def __init__(self, word_dim, hidden_dim=100, bptt_truncate=4):
        # Assign instance variables
        self.word_dim = word_dim
        self.hidden_dim = hidden_dim
        self.bptt_truncate = bptt_truncate
        # Randomly initialize the network parameters
        U = np.random.randn(hidden_dim, word_dim) * np.sqrt(2.0/word_dim)
        V = np.random.randn(word_dim, hidden_dim) * np.sqrt(2.0/hidden_dim)
        W = np.random.randn(hidden_dim, hidden_dim) * np.sqrt(2.0/hidden_dim)
        # Theano: Created shared variables
        self.U = theano.shared(name='U', value=U.astype(theano.config.floatX))
        self.V = theano.shared(name='V', value=V.astype(theano.config.floatX))
        self.W = theano.shared(name='W', value=W.astype(theano.config.floatX))
        # We store the Theano graph here
        self.theano = {}
        self.__theano_build__()
    
    def __theano_build__(self):
        U, V, W = self.U, self.V, self.W
        x = T.ivector('x')
        y = T.ivector('y')
        x_onehot = T.extra_ops.to_one_hot(x, self.word_dim)
        def forward_prop_step(x_t, s_t_prev, U, V, W):
            s_t = T.tanh(U.dot(x_t) + W.dot(s_t_prev))
            o_t = T.nnet.softmax(V.dot(s_t))
            return [o_t[0], s_t]
        [o,s], updates = theano.scan(
            forward_prop_step,
            sequences=x_onehot,
            outputs_info=[None, dict(initial=T.zeros(self.hidden_dim))],
            non_sequences=[U, V, W],
            truncate_gradient=self.bptt_truncate,
            strict=True)
        
        prediction = T.argmax(o, axis=1)
        o_error = T.sum(T.nnet.categorical_crossentropy(o, y))
        
        # Gradients
        dU = T.grad(o_error, U)
        dV = T.grad(o_error, V)
        dW = T.grad(o_error, W)
        
        # Assign functions
        self.forward_propagation = theano.function([x], o)
        self.predict = theano.function([x], prediction)
        self.ce_error = theano.function([x, y], o_error)
        self.bptt = theano.function([x, y], [dU, dV, dW])
        
        # SGD
        learning_rate = T.scalar('learning_rate')
        self.sgd_step = theano.function([x,y,learning_rate], [], 
                      updates=[(self.U, self.U - learning_rate * dU),
                              (self.V, self.V - learning_rate * dV),
                              (self.W, self.W - learning_rate * dW)])
    
    def calculate_loss(self, X, Y):
        return np.sum([self.ce_error(x,y) for x,y in zip(X,Y)])
    
    def calculate_mean_loss(self, X, Y):
        # Divide calculate_loss by the number of words
        num_words = np.sum([len(y) for y in Y])
        return self.calculate_loss(X,Y)/float(num_words)   


In [83]:
def gradient_check_theano(model, x, y, h=0.001, error_threshold=0.01):
    # Overwrite the bptt attribute. We need to backpropagate all the way to get the correct gradient
    model.bptt_truncate = 1000
    # Calculate the gradients using backprop
    bptt_gradients = model.bptt(x, y)
    # List of all parameters we want to chec.
    model_parameters = ['U', 'V', 'W']
    # Gradient check for each parameter
    for pidx, pname in enumerate(model_parameters):
        # Get the actual parameter value from the mode, e.g. model.W
        parameter_T = operator.attrgetter(pname)(model)
        parameter = parameter_T.get_value()
        print "Performing gradient check for parameter %s with size %d." % (pname, np.prod(parameter.shape))
        # Iterate over each element of the parameter matrix, e.g. (0,0), (0,1), ...
        it = np.nditer(parameter, flags=['multi_index'], op_flags=['readwrite'])
        while not it.finished:
            ix = it.multi_index
            # Save the original value so we can reset it later
            original_value = parameter[ix]
            # Estimate the gradient using (f(x+h) - f(x-h))/(2*h)
            parameter[ix] = original_value + h
            parameter_T.set_value(parameter)
            gradplus = model.calculate_loss([x],[y])
            parameter[ix] = original_value - h
            parameter_T.set_value(parameter)
            gradminus = model.calculate_loss([x],[y])
            estimated_gradient = (gradplus - gradminus)/(2*h)
            parameter[ix] = original_value
            parameter_T.set_value(parameter)
            # The gradient for this parameter calculated using backpropagation
            backprop_gradient = bptt_gradients[pidx][ix]
            # calculate The relative error: (|x - y|/(|x| + |y|))
            relative_error = np.abs(backprop_gradient - estimated_gradient)/(np.abs(backprop_gradient) + np.abs(estimated_gradient))
            # If the error is to large fail the gradient check
            if relative_error > error_threshold:
                print "Gradient Check ERROR: parameter=%s ix=%s" % (pname, ix)
                print "+h Loss: %f" % gradplus
                print "-h Loss: %f" % gradminus
                print "Estimated_gradient: %f" % estimated_gradient
                print "Backpropagation gradient: %f" % backprop_gradient
                print "Relative Error: %f" % relative_error
                return 
            it.iternext()
        print "Gradient check for parameter %s passed." % (pname)
        
np.random.seed(10)
# To avoid performing millions of expensive calculations we use a smaller vocabulary size for checking.
grad_check_vocab_size = 5
model = RNNTheano(grad_check_vocab_size, 10)
gradient_check_theano(model, [0,1,2,3], [1,2,3,4])

Performing gradient check for parameter U with size 50.
Gradient check for parameter U passed.
Performing gradient check for parameter V with size 50.
Gradient check for parameter V passed.
Performing gradient check for parameter W with size 100.
Gradient check for parameter W passed.


In [12]:
np.random.seed(10)
model = RNNTheano(vocabulary_size, 100, bptt_truncate=4)
# model.forward_propagation(X_train[3])[1][:10]
# model.predict(X_train[3])
# model.ce_error(X_train[3], y_train[3])
# print X_train[:5]

In [15]:
%timeit model.calculate_mean_loss(X_train[:100], y_train[:100])

1 loops, best of 3: 4.04 s per loop


In [93]:
def train_rnn_with_sgd_theano(model, X_train, y_train, learning_rate=0.0005, print_loss_after=5000, nepoch=1, anneal_after=-1, anneal_factor=0.5):
    # We keep track of the losses so we can plot them later
    losses = []
    num_examples_seen = 0
    for epoch in range(nepoch):
        for i in np.arange(len(y_train)):
            model.sgd_step(X_train[i], y_train[i], learning_rate)
            if (num_examples_seen % print_loss_after == 0):
                loss = model.calculate_mean_loss(X_train, y_train)
                losses.append(loss)
                print "Loss after epoch=%d i=%d: %f" % (epoch, i,loss)
            num_examples_seen += 1
        # Adjust the learning rate
        if(epoch % anneal_after == 0):
            learning_rate = learning_rate * anneal_factor

In [ ]:
np.random.seed(10)
model = RNNTheano(vocabulary_size, 100, bptt_truncate=4)
losses = train_rnn_with_sgd_theano(model, X_train[:5000], y_train[:5000], nepoch=100, learning_rate=0.005, anneal_after=5, print_loss_after=15000)